In [97]:
%%time
import pandas as pd

from sklearn.metrics import roc_auc_score, make_scorer, accuracy_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler


from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from xgboost import XGBClassifier
from sklearn.feature_selection import RFE


from time import strftime, strptime, mktime, time, ctime

from sklearn import model_selection

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.svm import LinearSVC

from xgboost import XGBClassifier
import xgboost as xgb

from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier,\
ExtraTreesClassifier, GradientBoostingClassifier, VotingClassifier, RandomForestClassifier

from mlxtend.classifier import StackingClassifier, StackingCVClassifier
from mlxtend.feature_selection import ColumnSelector

from sklearn.pipeline import make_pipeline
import numpy as np

import gc



CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 53.9 µs


In [2]:
%%time
df = pd.read_csv('./train.csv')
print df.shape
test = pd.read_csv('./test.csv')
print test.shape

y = df.pop('Approved')
train_id = df.pop('ID')
test_id = test.pop('ID')

def helper(x):
    try:
        tmp = strptime(x, '%d/%m/%y')
        tmp = mktime(tmp)
    except:
        tmp = 563000400
    return tmp

df['Lead_Creation_Date'] = df['Lead_Creation_Date'].apply(helper)
test['Lead_Creation_Date'] = test['Lead_Creation_Date'].apply(helper)

df['DOB'] = df['DOB'].apply(helper)
test['DOB'] = test['DOB'].apply(helper)

df = df.fillna(axis=1, value=-1)
test = test.fillna(axis=1, value=-1)

df = pd.get_dummies(df)
test = pd.get_dummies(test)
test = test.reindex(columns = df.columns, fill_value=0)

test_o = test.copy()

(69713, 22)
(30037, 21)
CPU times: user 13 s, sys: 6.96 s, total: 20 s
Wall time: 20 s


In [54]:
%%time

rid = []

for i in df:
    if df[i].value_counts().head(1).values[0]>df.shape[0]*0.996:
        rid.append(i)
print len(rid), df.shape[1]-len(rid)

37317 99
CPU times: user 33.9 s, sys: 0 ns, total: 33.9 s
Wall time: 33.9 s


### from 100 features

In [39]:
for i in set(df.columns)-set(rid):
    v = df[i].value_counts().head(1).values[0]
    if v>69300:
        print i, v

City_Code_C10022 69315
City_Code_C10023 69362
City_Code_C10024 69355
City_Code_C10025 69413
City_Code_C10026 69415
City_Code_C10027 69427
Customer_Existing_Primary_Bank_Code_B021 69312
Employer_Code_COM0000003 69389
Customer_Existing_Primary_Bank_Code_B023 69358
Customer_Existing_Primary_Bank_Code_B022 69325
Customer_Existing_Primary_Bank_Code_B027 69445
Customer_Existing_Primary_Bank_Code_B025 69386
Customer_Existing_Primary_Bank_Code_B024 69378
Source_S153 69310


In [40]:
update_rid = rid + ['Customer_Existing_Primary_Bank_Code_B027']
len(update_rid), len(rid)

(37317, 37316)

In [48]:
update_rid=rid

In [59]:
from kepcols import keep

In [60]:
update_rid = set(df.columns) - set(keep)

In [61]:
%%time

test = test_o.copy()
train = df.drop(update_rid, axis=1)
test = test.drop(update_rid, axis=1)

s = StandardScaler()
train = s.fit_transform(train)
test = s.transform(test)

print train.shape, test.shape

(69713, 99) (30037, 99)
CPU times: user 1.19 s, sys: 1.36 s, total: 2.54 s
Wall time: 2.54 s


### from here

In [117]:
%%time
m = lambda *x: [i.shape for i in x]
X_train, X_test, y_train, y_test = train_test_split(train, y, stratify=y, test_size=0.2, random_state=23333)
print m(X_train, y_train, X_test, y_test)

[(55770, 99), (55770,), (13943, 99), (13943,)]
CPU times: user 60 ms, sys: 22 ms, total: 82 ms
Wall time: 80.4 ms


In [118]:
%%time
d_train = xgb.DMatrix(X_train, label=y_train)
d_test = xgb.DMatrix(X_test, label=y_test)

CPU times: user 77 ms, sys: 18 ms, total: 95 ms
Wall time: 93.4 ms


In [119]:
RANDOM_STATE = 2018
params = {
    'min_child_weight': 0.8,
#     'eta': 0.1,
    'colsample_bytree': 0.8,
    'max_depth': 12,
    'subsample': 0.8,
    'alpha': 0.3,
    'lambda':0.9,
    'gamma': 0.3,
    'silent': 0,
    'learning_rate':0.02,
    'verbose_eval': True,
    'seed': RANDOM_STATE,
    'objective':"binary:logistic",
#     'eval_metric':'error'
#     'num_class':1
    'n_jobs':24,
    'eval_metric':'auc'
    
}
watchlist = [(d_train, 'train'), (d_test, 'test')]



In [120]:
start=time()

clf = xgb.train(params, d_train, 10000, watchlist, early_stopping_rounds=200, 
                verbose_eval=50,  maximize=True,
               )
                


'!!! used %.2f s'%(time()-start)

[0]	train-auc:0.746156	test-auc:0.75242
Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 200 rounds.
[50]	train-auc:0.86644	test-auc:0.836967
[100]	train-auc:0.915329	test-auc:0.845848
[150]	train-auc:0.953751	test-auc:0.849026
[200]	train-auc:0.974008	test-auc:0.853793
[250]	train-auc:0.98517	test-auc:0.856178
[300]	train-auc:0.990492	test-auc:0.857274
[350]	train-auc:0.994263	test-auc:0.85668
[400]	train-auc:0.996361	test-auc:0.856076
[450]	train-auc:0.997762	test-auc:0.855154
Stopping. Best iteration:
[288]	train-auc:0.989359	test-auc:0.857571



'!!! used 14.00 s'

In [121]:
%%time
dtest = xgb.DMatrix(test)
predb = clf.predict(dtest)

submit = pd.concat([pd.DataFrame(test_id.values, columns=['ID']), pd.DataFrame(predb, columns=['Approved'])], axis=1)
submit.shape

CPU times: user 2.09 s, sys: 8 ms, total: 2.1 s
Wall time: 132 ms


___

(30037, 2)

In [122]:
submit.to_csv('./submit/xgb_rm996_23333.csv',index=False)

#### lb .852388

In [60]:
# clf.save_model('xgb_0.852586.model')

In [441]:
roc_auc_score?

In [442]:
submit2 = pd.concat([pd.DataFrame(test_id.values, columns=['ID']), pd.DataFrame(pred, columns=['Approved'])], axis=1)
submit2.shape

(30037, 2)

In [449]:
accuracy_score(y_test, [0]*y_test.shape[0])

0.9834325467976762

___

In [276]:
cross_val_score(RandomForestClassifier(n_estimators=50, n_jobs=24), df.iloc[:,:-1], df.iloc[:,-1], cv=5)

array([0.87807502, 1.        , 0.99992828, 1.        , 0.98759145])

In [76]:
fr = RandomForestClassifier(n_estimators=50)
fr

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [77]:
fr.fit(df.iloc[:,:-1], df.iloc[:,-1])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [ ]:
fr.predict_log_proba